In [100]:
# %pip install py7zr
# %pip install io
# %pip install schedule
# %pip install openpyxl

In [101]:
import requests, pandas as pd, io, zipfile, os, py7zr, copy, ast

In [102]:
## Download and extract .7z file from url
# URL to the .7z file
url = "https://unctadstat-api.unctad.org/bulkdownload/US.CommodityPrice_M/US_CommodityPrice_M"

# Define a path for saving the downloaded .7z file
file_path = os.path.join(os.getcwd(), 'downloaded_data.7z')

# Send a GET request to the URL
response = requests.get(url, stream=True)  # Stream=True to avoid loading the content into memory at once

# Check if the request was successful
if response.status_code == 200:
    # Write the content to the .7z file
    with open(file_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192): 
            if chunk:  # filter out keep-alive new chunks
                f.write(chunk)

    # Try to extract the .7z file
    with py7zr.SevenZipFile(file_path, mode='r') as z:
        z.extractall(path=os.getcwd())
    print("Extraction complete")
else:
    print(f"Download failed with status code: {response.status_code}")
    

## Retrieves CSV file and saves as DF
    
# List all files in the current directory
extracted_files = os.listdir(os.getcwd())

# You might want to filter for a specific file type, e.g., CSV files
csv_files = [file for file in extracted_files if file.endswith('.csv')]

# Assuming you want to work with the first CSV file
if csv_files:
    df = pd.read_csv(csv_files[0])
    print(df.head())
else:
    print("No CSV files found in the extracted contents.")


Extraction complete
    Period Period Label  Commodity  \
0  1995M01    Jan. 1995  IN0001.20   
1  1995M01    Jan. 1995  IN0001.01   
2  1995M01    Jan. 1995  IN0001.02   
3  1995M01    Jan. 1995  080300.01   
4  1995M01    Jan. 1995  020100.01   

                                     Commodity Label  Prices Prices Footnote  \
0                                         All groups     NaN             NaN   
1                                           All food     NaN             NaN   
2                                               Food     NaN             NaN   
3  Bananas, Central and South America, FOT, US im...    0.35             NaN   
4  Beef, Australia/New Zealand, frozen, CIF US po...    2.12             NaN   

  Prices Missing value  
0       Not applicable  
1       Not applicable  
2       Not applicable  
3                  NaN  
4                  NaN  


In [103]:
df.head()

,Period,Period Label,Commodity,Commodity Label,Prices,Prices Footnote,Prices Missing value
0,1995M01,Jan. 1995,IN0001.20,All groups,NaN,NaN,Not applicable
1,1995M01,Jan. 1995,IN0001.01,All food,NaN,NaN,Not applicable
2,1995M01,Jan. 1995,IN0001.02,Food,NaN,NaN,Not applicable
3,1995M01,Jan. 1995,080300.01,"Bananas, Central and South America, FOT, US im...",0.35,NaN,NaN
4,1995M01,Jan. 1995,020100.01,"Beef, Australia/New Zealand, frozen, CIF US po...",2.12,NaN,NaN


In [104]:
df.iloc[-1]

Period                                                            2023M12
Period Label                                                    Dec. 2023
Commodity                                                       271100.01
Commodity Label         Natural gas, index, Europe, United States and ...
Prices                                                              95.16
Prices Footnote                                                       NaN
Prices Missing value                                                  NaN
Name: 19015, dtype: object

In [105]:
df[df["Prices Missing value"]=="Not applicable"]["Commodity Label"].unique()

array(['All groups', 'All food', 'Food', 'Tropical beverages',
       'Vegetable oilseeds and oils', 'Agricultural raw materials',
       'Minerals, ores and metals', 'Fuels'], dtype=object)

In [106]:
groups = df["Commodity Label"].unique()
print(len(groups))
[print("Group : " + i) for i in groups]

58
Group : All groups
Group : All food
Group : Food
Group : Bananas, Central and South America, FOT, US import price ($/kg)
Group : Beef, Australia/New Zealand, frozen, CIF US ports ($/kg)
Group : Maize, Argentina, Rosario, Up River, FOB ($/t)
Group : Maize, United States, n° 3 yellow, FOB Gulf ($/t)
Group : Rice, Thailand, white milled, 5% broken, export price, FOB ($/t)
Group : Salmon, fresh, fish-farm bred, export price, Norway ($/kg)
Group : Shrimps, brown, no. 1, shell-on, headless, Mexico ($/kg)
Group : Soybean meal, in bulk, 45/46% protein, Hamburg FOB ex-mill ($/t)
Group : Sugar, average of I.S.A. daily prices, FOB & Stowed, in bulk, Caribbean ports (¢/lb.)
Group : Wheat, Argentina, Trigo Pan, Up River, FOB ($/t)
Group : Wheat, United States, n° 2 Hard Red Winter (ordinary), FOB Gulf ($/t)
Group : Tropical beverages
Group : Cocoa beans, average daily prices New York/London (¢/lb.)
Group : Coffee, Brazilian and other natural Arabicas, ex-dock USA (¢/lb.)
Group : Coffee, Colombia

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [107]:
df = df[["Period", "Commodity Label", "Prices"]]
df

,Period,Commodity Label,Prices
0,1995M01,All groups,NaN
1,1995M01,All food,NaN
2,1995M01,Food,NaN
3,1995M01,"Bananas, Central and South America, FOT, US im...",0.35
4,1995M01,"Beef, Australia/New Zealand, frozen, CIF US po...",2.12
...,...,...,...
19011,2023M12,"Zinc, Prime Western, delivered, North America ...",135.00
19012,2023M12,Fuels,NaN
19013,2023M12,"Crude oil, Dubai, medium, Fateh API 32°, spot ...",77.22
19014,2023M12,"Crude oil, UK Brent, light blend API 38°, spot...",77.85


In [108]:
us_groups = [group for group in groups if "US" in group or "United States" in group or "North America" in group]
us_groups = [value for index, value in enumerate(us_groups) if index not in [0, 1, 4, 5, 6, 7, 10, 11, 13]]
us_groups

['Maize, United States, n° 3 yellow, FOB Gulf ($/t)',
 'Wheat, United States, n° 2 Hard Red Winter (ordinary), FOB Gulf ($/t)',
 'Soybeans, in bulk, United States, n° 2 yellow, CIF Rotterdam ($/t)',
 "Cattle hides, US Chicago packer's heavy native steers, FOB shipping point (¢/lb.)",
 'Natural gas, index, Europe, United States and Japan (2010=100)']

In [109]:
df = df[df["Commodity Label"].isin(us_groups)].reset_index(drop=True)
df = df.pivot_table(index='Period', columns='Commodity Label', values='Prices', aggfunc='first').reset_index()
df.columns.name = None
df.rename(columns={"Period": "Date", 
                    df.columns[1]: "Cattle Hide (Chicago)",
                    df.columns[2]: "Maize (US)",
                    df.columns[3]: "Natural Gas (US)",
                    df.columns[4]: "Soybeans (US)",
                    df.columns[5]: "Wheat (US)"
                   }, inplace=True)

In [110]:
commodity_prices = copy.deepcopy(df)
commodity_prices.head()

,Date,Cattle Hide (Chicago),Maize (US),Natural Gas (US),Soybeans (US),Wheat (US)
0,1995M01,90.12,109.1,33.25,243.0,159.8
1,1995M02,91.42,108.1,34.13,238.0,155.2
2,1995M03,97.95,110.3,33.64,246.0,149.8
3,1995M04,102.32,111.6,34.89,249.0,151.0
4,1995M05,99.61,113.6,35.18,250.0,162.7


In [111]:
xlsx_file_path = "C:/Users/tyson/Downloads/longexp1/2024M01_IPP_HIS_BEAExportHistory.xlsx"
df_bea = pd.read_excel(xlsx_file_path)

c:\Users\tyson\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [112]:
selected_columns = [df_bea.columns[1]] + list(df_bea.columns[4:16])
df_bea = df_bea[selected_columns]

new_header = df_bea.iloc[3]  # Select the row you want as the new header
df_bea = df_bea[5:3668]  # Take the data less the header row
df_bea.columns = new_header  # Set the new header
df_bea.rename(columns={df_bea.columns[0]:"Year/Commodity"}, inplace=True) 
df_bea.columns.name = None
df_bea.reset_index(drop=True, inplace=True)  # Reset the index

df_bea.head(50)

,Year/Commodity,Jan.,Feb.,Mar.,Apr.,May,Jun.,Jul.,Aug.,Sep.,Oct.,Nov.,Dec.
0,All commodities,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1983,–,–,–,–,–,–,–,–,85.7,–,–,85.4
2,1984,–,–,85.9,–,–,87.1,–,–,85.1,–,–,84.2
3,1985,–,–,84,–,–,84.4,–,–,83.7,–,–,83.8
4,1986,–,–,83.6,–,–,83.3,–,–,82.4,–,–,83.3
5,1987,–,–,84.3,–,–,86,–,–,86.6,–,–,88.3
6,1988,–,–,89.8,–,–,92.2,–,–,94.1,–,–,93.8
7,1989,94.9,94.6,95.2,95,95.4,95.2,94.7,94,94.4,94.6,94.2,94.4
8,1990,95.2,94.6,94.8,95.1,95.3,95.1,95.3,95.4,95.8,96.6,96.7,96.3
9,1991,96.8,96.7,96.4,96.6,96.4,96.1,95.7,96,95.6,96.6,96.7,95.7


In [113]:
def safe_literal_eval(value):
    try:
        return ast.literal_eval(value)
    except (ValueError, SyntaxError):
        # Handle the error or return the original value
        return value

df_bea["Year/Commodity"] = df_bea["Year/Commodity"].apply(safe_literal_eval)

groups = df_bea[df_bea['Year/Commodity'].apply(lambda x: isinstance(x, str))]["Year/Commodity"].unique()
years = df_bea[df_bea['Year/Commodity'].apply(lambda x: isinstance(x, int))]["Year/Commodity"].unique()

years.sort()

df = pd.DataFrame(columns=groups)
df['Date'] = [f"{year}M{str(month).zfill(2)}" for year in years for month in range(1, 13)]
last_col = df.iloc[:, -1]
df = df.iloc[:, :-1]
df.insert(0, last_col.name, last_col)
df = df.drop('Advanced Technology', axis=1)

df

,Date,All commodities,Agricultural commodities,All exports excluding food and fuels (Dec. 2010=100),All exports excluding fuels (June 2022=100),Nonagricultural commodities,"All commodities, excluding computers, & semiconductors (Dec. 2001=100)","Foods, feeds, & beverages","Agricultural foods, feeds & beverages, excluding distilled beverages",Wheat and rice,...,Jewelry (Dec. 2022=100),Nonmanufactured consumer goods (Dec. 2018=100),Consumer durables--nonmanufactured (Dec. 2023=100),Gem diamonds (Dec. 2023=100),Biotechnology (Dec. 2021=100),Life Science (Dec. 2019=100),Opto-Electronics (Dec. 2022=100),Information & Communications (Dec. 2019=100),Electronics (Dec. 2019=100),Aerospace (Dec. 2019=100)
0,1978M01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1978M02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1978M03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1978M04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1978M05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
559,2024M08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
560,2024M09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
561,2024M10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
562,2024M11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [114]:
commodity_years = {}
for i in df_bea["Year/Commodity"]:
    if isinstance(i, str):
        commodity_years[i] = []
        current = i
    elif isinstance(i, int):
        commodity_years[current].append(i)
    else:
        pass

removed_value = commodity_years.pop('Advanced Technology', None)

In [115]:
month_dict_of_lists = {}
for month in df_bea.columns[1:16]:
    list_of_lists = []
    current_list = []

    for val in df_bea[month]:
        if pd.notna(val):
            current_list.append(val)
        else:
            if current_list:  # if current_list is not empty
                list_of_lists.append(current_list)
                current_list = []

    # Don't forget to add the last list if the last value is not NaN
    if current_list:
        list_of_lists.append(current_list)
    
    month_dict_of_lists[month] = list_of_lists


In [116]:
# combined = {}
# for commodity, years in commodity_years.items():
#     combined[commodity] = {}
#     for i, year in enumerate(years):
#         combined[commodity][year] = {}
#         for month in month_dict_of_lists:
#             # Assume that the length of the months_values[month] matches the length of the years list
#             combined[commodity][year][month] = month_dict_of_lists[month][i]

# combined

In [117]:
# # Assuming the structure of your dictionaries is as follows:
# # commodity_years = {'Commodity1': [1990, 1991, 1992], 'Commodity2': [1990, 1991, 1992]}
# # months_values = {'Jan': [10, 20, 30], 'Feb': [15, 25, 35], ..., 'Dec': [50, 60, 70]}

# # Initialize the list that will hold all the flattened data
# flattened_data = []

# # Loop over each commodity and its corresponding years
# for commodity, years in commodity_years.items():
#     for year in years:
#         # For each year, create a new dictionary for that commodity and year
#         year_data = {}
#         for month, values in month_dict_of_lists.items():
#             # Construct the year-month key (e.g., "1990Jan")
#             year_month_key = f"{year}{month}"
#             # Fetch the correct value for the month and year. 
#             # The index for the year in 'years' list is used to fetch the corresponding value from the 'values' list
#             index = years.index(year)
#             year_data[year_month_key] = values[index]
        
#         # Add the dictionary for this commodity and year to the flattened data list
#         flattened_data.append({commodity: year_data})

# # 'flattened_data' now contains a list of dictionaries for each commodity-year combination with month values
# flattened_data

In [118]:
# # Assume months are ordered correctly in the months_values dictionary
# months_order = {
#     'Jan.': 'M01', 'Feb.': 'M02', 'Mar.': 'M03', 'Apr.': 'M04', 'May': 'M05', 'Jun.': 'M06',
#     'Jul.': 'M07', 'Aug.': 'M08', 'Sep.': 'M09', 'Oct.': 'M10', 'Nov.': 'M11', 'Dec.': 'M12'
# }

# flattened_data = []

# for commodity, years in commodity_years.items():
#     for year in years:
#         year_data = {}
#         for month, values in month_dict_of_lists.items():
#             # Use the month's order number instead of the name
#             month_number = months_order[month]  # Convert month name to 'M##' format
#             year_month_key = f"{year}{month_number}"
#             index = years.index(year)  # The index for the year is used to fetch the corresponding month value
#             year_data[year_month_key] = values[index]
        
#         # Add the dictionary for this commodity and year to the flattened data list
#         flattened_data.append({commodity: year_data})

# # Now 'flattened_data' contains the correctly formatted keys
# flattened_data


In [119]:
# flattened_data[5]

In [120]:
# # Let's assume 'flattened_data' is a list of dictionaries as from previous steps, 
# # and 'df' is the DataFrame you've provided.

# # First, we will convert 'flattened_data' into a more accessible form:
# flattened_dict = {}
# for data in flattened_data:
#     for commodity, year_data in data.items():
#         for year_month, value in year_data.items():
#             if commodity not in flattened_dict:
#                 flattened_dict[commodity] = {}
#             flattened_dict[commodity][year_month] = value

# # Now, iterate over the DataFrame and update values
# for index, row in df.iterrows():
#     # Extract the year_month from the 'Date' column
#     year_month = row['Date']
#     # Update each commodity column with the corresponding value if it exists
#     for commodity in df.columns[1:]:  # Skipping the 'Date' column
#         if commodity in flattened_dict and year_month in flattened_dict[commodity]:
#             df.at[index, commodity] = flattened_dict[commodity][year_month]

# # After this, 'df' should have the updated values where they match by year-month
# df

In [121]:
# # Convert month names to 'M##' format
# month_mapping = {
#     'Jan.': 'M01', 'Feb.': 'M02', 'Mar.': 'M03', 'Apr.': 'M04', 'May': 'M05', 'Jun.': 'M06',
#     'Jul.': 'M07', 'Aug.': 'M08', 'Sep.': 'M09', 'Oct.': 'M10', 'Nov.': 'M11', 'Dec.': 'M12'
# }

# # Prepare a DataFrame to hold all the data
# df_final = pd.DataFrame()

# # Process each commodity
# for commodity, years in commodity_years.items():
#     for i, year in enumerate(years):
#         for month, month_values in month_dict_of_lists.items():
#             # Translate the month name to the 'M##' format
#             month_code = month_mapping[month]
#             # Create the 'YearM##' format
#             year_month = f"{year}{month_code}"
#             # Get the list of values for this particular year and month
#             values = month_values[i]
#             # Populate the DataFrame
#             for value in values:
#                 # Append a new row with the YearM##, Commodity and Value
#                 df_final = df_final.append({'Date': year_month, commodity: value}, ignore_index=True)

# # Ensure the 'Date' column is in the front
# cols = ['Date'] + [col for col in df_final if col != 'Date']
# df_final = df_final[cols]

# # Now df_final is the DataFrame with the correct YearM## and values for each commodity
# df_final

In [122]:
# # Convert month names to 'M##' format
# month_mapping = {
#     'Jan.': 'M01', 'Feb.': 'M02', 'Mar.': 'M03', 'Apr.': 'M04', 'May': 'M05', 'Jun.': 'M06',
#     'Jul.': 'M07', 'Aug.': 'M08', 'Sep.': 'M09', 'Oct.': 'M10', 'Nov.': 'M11', 'Dec.': 'M12'
# }

# # Prepare a list to collect all the data entries
# data_entries = []

# # Process each commodity
# for commodity, years in commodity_years.items():
#     for i, year in enumerate(years):
#         for month, month_values in month_dict_of_lists.items():
#             # Translate the month name to the 'M##' format
#             month_code = month_mapping[month]
#             # Create the 'YearM##' format
#             year_month = f"{year}{month_code}"
#             # Get the list of values for this particular year and month
#             values = month_values[i]
#             # Populate the list with data entries
#             for value in values:
#                 # Each entry is a dictionary with the Date, Commodity and Value
#                 data_entries.append({'Date': year_month, commodity: value})

# # Now create a DataFrame from the list of data entries
# df_final = pd.DataFrame(data_entries)

# # Reorder the DataFrame columns if necessary
# df_final = df_final[['Date'] + [col for col in df_final.columns if col != 'Date']]

# # Now df_final is the DataFrame with the correct YearM## and values for each commodity
# df_final

In [123]:
# # Convert the list of dictionaries to a DataFrame
# df_entries = pd.DataFrame(data_entries)

# # Now, pivot the DataFrame to get unique year-month combinations as rows and commodities as columns
# # with the values as the cell data
# df_final = df_entries.pivot_table(index='Date', columns='Commodity', values='Value', aggfunc='first')

# # Reset the index to turn the 'Date' index back into a column
# df_final.reset_index(inplace=True)

# # If necessary, sort the DataFrame based on 'Date'
# df_final = df_final.sort_values('Date')


In [124]:
# # Initialize an empty list to hold the data
# data_entries = []

# # Convert the month names to 'M##' format
# month_mapping = {
#     'Jan.': 'M01', 'Feb.': 'M02', 'Mar.': 'M03', 'Apr.': 'M04', 'May.': 'M05', 'Jun.': 'M06',
#     'Jul.': 'M07', 'Aug.': 'M08', 'Sep.': 'M09', 'Oct.': 'M10', 'Nov.': 'M11', 'Dec.': 'M12'
# }

# # Get a list of all commodities
# commodities = list(commodity_years.keys())

# # Iterate over each month and its list of lists
# for month, values_list in month_dict_of_lists.items():
#     month_code = month_mapping[month]
#     # For each sub-list in the list of lists
#     for i, yearly_values in enumerate(values_list):
#         # Match each value to its corresponding commodity and year
#         for j, value in enumerate(yearly_values):
#             commodity = commodities[j]
#             year = commodity_years[commodity][i]
#             year_month = f"{year}{month_code}"
#             # Append a dictionary for each commodity-year-month combination
#             data_entries.append({'Date': year_month, 'Commodity': commodity, 'Value': value})

# # Convert the list of dictionaries to a DataFrame
# df_entries = pd.DataFrame(data_entries)

# # Pivot this DataFrame to get the final structure
# df_final = df_entries.pivot_table(index='Date', columns='Commodity', values='Value', aggfunc='first')

# # Reset the index to turn the 'Date' index back into a column
# df_final.reset_index(inplace=True)

# # Now df_final is the DataFrame with the correct YearM## and values for each commodity


In [125]:
# # Convert the month names to 'M##' format
# month_mapping = {
#     'Jan.': 'M01', 'Feb.': 'M02', 'Mar.': 'M03', 'Apr.': 'M04', 'May': 'M05', 'Jun.': 'M06',
#     'Jul.': 'M07', 'Aug.': 'M08', 'Sep.': 'M09', 'Oct.': 'M10', 'Nov.': 'M11', 'Dec.': 'M12'
# }

# # Prepare a list to collect all the data entries
# data_entries = []

# # Process each commodity
# for commodity, years in commodity_years.items():
#     for i, year in enumerate(years):
#         for month, month_values in month_dict_of_lists.items():
#             # Translate the month name to the 'M##' format
#             month_code = month_mapping[month]
#             # Create the 'YearM##' format
#             year_month = f"{year}{month_code}"
#             # Get the list of values for this particular year and month
#             values = month_values[i]
#             # Populate the list with data entries
#             for value in values:
#                 # Each entry is a dictionary with the Date, Commodity and Value
#                 data_entries.append({'Date': year_month, 'Commodity': commodity, 'Value': value})

# # Convert the list of dictionaries to a DataFrame
# df_entries = pd.DataFrame.from_records(data_entries)

# # Pivot this DataFrame to get the final structure
# df_final = df_entries.pivot_table(index='Date', columns='Commodity', values='Value', aggfunc='first')

# # Reset the index to turn the 'Date' index back into a column
# df_final.reset_index(inplace=True)

# # Now df_final is the DataFrame with the correct YearM## and values for each commodity
# df_final

In [126]:
# Convert month names to 'M##' format
month_mapping = {
    'Jan.': 'M01', 'Feb.': 'M02', 'Mar.': 'M03', 'Apr.': 'M04', 'May': 'M05', 'Jun.': 'M06',
    'Jul.': 'M07', 'Aug.': 'M08', 'Sep.': 'M09', 'Oct.': 'M10', 'Nov.': 'M11', 'Dec.': 'M12'
}

# Initialize a list to store the data for DataFrame
data_for_df = []

# Iterate over the commodities and years
for commodity, years in commodity_years.items():
    # Iterate over the months and their values
    for month, values in month_dict_of_lists.items():
        month_code = month_mapping[month]
        # For each year, get the value from the corresponding list in the values
        for i, year in enumerate(years):
            # Create the 'YearM##' format
            year_month = f"{year}{month_code}"
            # Get the corresponding value for this month and year
            value = values[i]
            # Add a new record to the list
            data_for_df.append((year_month, commodity, value))

# Now, create a DataFrame with all the data
df = pd.DataFrame(data_for_df, columns=['Date', 'Commodity', 'Value'])

# Pivot the DataFrame to have 'Date' as the index and commodities as columns
# Since there should be no duplicate 'Date' entries for the same 'Commodity',
# we should not encounter the 'ValueError' this time.
df_pivoted = df.pivot(index='Date', columns='Commodity', values='Value')

# Reset the index to turn 'Date' back into a column
df_pivoted = df_pivoted.reset_index()


In [127]:
df_pivoted

Commodity,Date,Aerospace (Dec. 2019=100),Agricultural commodities,"Agricultural foods, feeds & beverages, excluding distilled beverages",Agricultural industrial supplies & materials,Agricultural machinery and equipment (Dec. 2016=100),"Agricultural machinery and equipment, 5 digit (Dec. 2023=100)",All commodities,"All commodities, excluding computers, & semiconductors (Dec. 2001=100)",All exports excluding food and fuels (Dec. 2010=100),...,"Telecommunications equipment, 5 digit (Dec. 2023=100)",Toiletries & cosmetics,"Toys, shooting and sporting goods",Transportation equipment excluding motor vehicles (Dec. 2001=100),"Trucks, buses, & special purpose vehicles (Dec. 2023=100)",Vegetables and vegetable preparations and juices (Dec. 2018=100),Vehicles designed to transport goods,Vehicles-passenger (Dec. 2023=100),Wheat,Wheat and rice
0,1978M01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1978M02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1978M03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1978M04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1978M05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
559,2024M08,"[–, 101.8, 103.7, 108.8, 113.5, 120.8, 126.3, ...","[–, –, –, –, –, –, –, –, –, –, –, –, –, 95.3, ...","[–, –, –, –, –, –, –, –, –, –, 137.2, 137.3, 1...","[–, –, –, –, –, –, –, –, –, –, 137.2, 137.3, 1...","[–, –, –, –, –, –, –, –, –, –, 104.5, 147, 157...","[–, –, –, –, 108.2, 103.5, 103.4, 99.6, 105.8,...","[–, –, –, –, –, –, –, –, –, –, 137.2, 137.3, 1...","[–, 108.6, 118, 118.2, 102.9, 124.9, 124.3, 10...","[–, –]",...,"[–, –, –, –, 108.2, 103.5, 103.4, 99.6, 105.8,...","[–, –, –, –, 99.4, 102.8, 109.4, 104.3, 105.3,...","[–, –]","[–, 108.6, 118, 118.2, 102.9, 124.9, 124.3, 10...","[–, –, –, –, 108.2, 103.5, 103.4, 99.6, 105.8,...","[–, –, –, –, –, –, 106.3, 100.6, 102.2, 97.1, ...","[–, –, –, –, –, –, –, –, –, –, 68.4, 71.5, 71....","[–, –, –, –, 108.2, 103.5, 103.4, 99.6, 105.8,...","[–, –, –, –, –, –, –, –, –, –, –, 92.7, 107.7,...","[–, 184.1, 233.9, 291.7, 242.8, 232.5, 236.2, ..."
560,2024M09,"[–, 102.2, 104.2, 109.2, 114.7, 120.3, 126.8, ...","[80.5, 63.1, 89.7, 73.6, 63.7, 68.3, 91.6, 119...","[–, 117.2, 147.5, 166.6, 155.6, 130.8, 141, 12...","[–, 117.2, 147.5, 166.6, 155.6, 130.8, 141, 12...","[–, 109.7, 83.7, 83.3, 124.2, 133.4, 98.4, 92....","[94.2, 83.1, 89, 116.5, 107.5, 103.4, 102.2, 1...","[–, 117.2, 147.5, 166.6, 155.6, 130.8, 141, 12...","[–, 109.1, 119.6, 120.6, 104.5, 128.4, 126.5, ...","[–, –]",...,"[94.2, 83.1, 89, 116.5, 107.5, 103.4, 102.2, 1...","[93.7, 94, 90, 94.2, 98.5, 103.2, 109.1, 104.9...","[–, –]","[–, 109.1, 119.6, 120.6, 104.5, 128.4, 126.5, ...","[94.2, 83.1, 89, 116.5, 107.5, 103.4, 102.2, 1...","[120.3, 106.9, 93.3, 83.8, 84.5, 119.6, 105.9,...","[80, 79.9, 79.5, 62.7, 67.7, 65.1, 72.4, 80.8,...","[94.2, 83.1, 89, 116.5, 107.5, 103.4, 102.2, 1...","[77.1, 77.1, 75.8, 71.6, 84, 99.8, 90.8, 93, 8...","[–, 190.5, 241.8, 283.4, 248.8, 237.6, 233.8, ..."
561,2024M10,"[–, 102.2, 104.6, 110.1, 115.7, 119.9, 127.9, ...","[–, –, –, –, –, –, –, –, –, –, –, –, –, 99.8, ...","[–, –, –, –, –, –, –, –, –, –, 139.8, 136.4, 1...","[–, –, –, –, –, –, –, –, –, –, 139.8, 136.4, 1...","[–, –, –, –, –, –, –, –, –, –, 123.9, 158.9, 1...","[–, –, –, –, 107.1, 101.3, 105.8, 100, 105, 10...","[–, –, –, –, –, –, –, –, –, –, 139.8, 136.4, 1...","[–, 111.7, 119.5, 122.1, 108.6, 131.8, 126.4, ...","[–, –]",...,"[–, –, –, –, 107.1, 101.3, 105.8, 100, 105, 10...","[–, –, –, 93.1, 97.5, 104.2, 109, 104.7, 103.7...","[–, –]","[–, 111.7, 119.5, 122.1, 108.6, 131.8, 126.4, ...","[–, –, –, –, 107.1, 101.3, 105.8, 100, 105, 10...","[–, –, –, –, –, –, 104.9, 97.7, 104.8, 97.4, 1...","[–, –, –, –, –, –, –, –, –, 70, 68.3, 69.2, 72...","[–, –, –, –, 107.1,

In [128]:
df_2021 = df_final[df_final['Date'].str.contains("2021")]
df_2021

,Date,All commodities,Agricultural commodities,All exports excluding food and fuels (Dec. 2010=100),All exports excluding fuels (June 2022=100),Nonagricultural commodities,"All commodities, excluding computers, & semiconductors (Dec. 2001=100)","Foods, feeds, & beverages","Agricultural foods, feeds & beverages, excluding distilled beverages",Wheat and rice,...,Jewelry (Dec. 2022=100),Nonmanufactured consumer goods (Dec. 2018=100),Consumer durables--nonmanufactured (Dec. 2023=100),Gem diamonds (Dec. 2023=100),Biotechnology (Dec. 2021=100),Life Science (Dec. 2019=100),Opto-Electronics (Dec. 2022=100),Information & Communications (Dec. 2019=100),Electronics (Dec. 2019=100),Aerospace (Dec. 2019=100)


In [129]:
df_bea.head(50)

,Year/Commodity,Jan.,Feb.,Mar.,Apr.,May,Jun.,Jul.,Aug.,Sep.,Oct.,Nov.,Dec.
0,All commodities,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1983,–,–,–,–,–,–,–,–,85.7,–,–,85.4
2,1984,–,–,85.9,–,–,87.1,–,–,85.1,–,–,84.2
3,1985,–,–,84,–,–,84.4,–,–,83.7,–,–,83.8
4,1986,–,–,83.6,–,–,83.3,–,–,82.4,–,–,83.3
5,1987,–,–,84.3,–,–,86,–,–,86.6,–,–,88.3
6,1988,–,–,89.8,–,–,92.2,–,–,94.1,–,–,93.8
7,1989,94.9,94.6,95.2,95,95.4,95.2,94.7,94,94.4,94.6,94.2,94.4
8,1990,95.2,94.6,94.8,95.1,95.3,95.1,95.3,95.4,95.8,96.6,96.7,96.3
9,1991,96.8,96.7,96.4,96.6,96.4,96.1,95.7,96,95.6,96.6,96.7,95.7


In [133]:
df_bea = df_bea[(df_bea['Year/Commodity'] == 2022) | df_bea['Year/Commodity'].apply(lambda x: isinstance(x, str))]
df_bea = df_bea[df_bea['Year/Commodity'] != "Advanced Technology"]
df_bea['Year/Commodity'] = df_bea['Year/Commodity'].astype(str)
df_bea = df_bea[~df_bea['Year/Commodity'].str.contains("\(Dec\. 2023=100\)")]
df_bea.reset_index(drop=True, inplace=True)
df_bea

[print(i) for i in df_bea['Year/Commodity']]

All commodities
2022
Agricultural commodities
2022
All exports excluding food and fuels  (Dec. 2010=100)
2022
All exports excluding fuels   (June 2022=100)
2022
Nonagricultural commodities
2022
All commodities, excluding computers, & semiconductors  (Dec. 2001=100)
2022
Foods, feeds, & beverages
2022
Agricultural foods, feeds & beverages, excluding distilled beverages
2022
Wheat and rice
2022
Wheat
2022
Soybeans & other oil seeds
2022
Soybeans and soybean by-products, prior to the extraction of oil
2022
Feedstuff
2022
Corn
2022
Other agricultural foods
2022
Meat, poultry & other edible animal products (Dec. 2006=100)
2022
Fruit and fruit preparations, including fruit juices
2022
Vegetables and vegetable preparations and juices (Dec. 2018=100)
2022
Nuts & preparations   (Dec. 2014=100)
2022
Bakery & confectionery products   (Dec. 2016=100)
2022
Other foods and food preparations (lard, soft bev, spices), n.e.s.
2022
Nonagricultural foods (fish, distilled beverages)
2022
Fish and shellfis

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [136]:
# Extract the headers (commodities) and create a mapping for the new headers
commodities = df_bea[df_bea['Year/Commodity'] != '2022']['Year/Commodity'].dropna().tolist()
headers_mapping = {i: commodity for i, commodity in enumerate(commodities, start=1)}

# Filter out the rows where 'Year/Commodity' is not a year
df_bea = df_bea[df_bea['Year/Commodity'] == '2022'].reset_index(drop=True)

# Drop the 'Year/Commodity' column as we no longer need it
df_bea.drop('Year/Commodity', axis=1, inplace=True)

# Rename the columns using the headers_mapping created earlier
df_bea.rename(columns=headers_mapping, inplace=True)

# Transpose the DataFrame so that months become rows and commodities become columns
df_bea = df_bea.T

# Reset the index to turn the old column names into a column in the DataFrame
df_bea.reset_index(inplace=True)

# Rename the columns appropriately
df_bea.columns = ['Month'] + commodities

# The 'Month' column will have values from the index (Jan, Feb, etc.), extract and set it
df_bea['Month'] = df_bea['Month'].str.extract('(\w+)', expand=False)

# Now, df should have the structure you want
df_bea


,Month,All commodities,Agricultural commodities,All exports excluding food and fuels (Dec. 2010=100),All exports excluding fuels (June 2022=100),Nonagricultural commodities,"All commodities, excluding computers, & semiconductors (Dec. 2001=100)","Foods, feeds, & beverages","Agricultural foods, feeds & beverages, excluding distilled beverages",Wheat and rice,...,"Toys, shooting and sporting goods","Coins, gems, jewelry & collectibles (Dec. 2022=100)",Jewelry (Dec. 2022=100),Nonmanufactured consumer goods (Dec. 2018=100),Biotechnology (Dec. 2021=100),Life Science (Dec. 2019=100),Opto-Electronics (Dec. 2022=100),Information & Communications (Dec. 2019=100),Electronics (Dec. 2019=100),Aerospace (Dec. 2019=100)
0,Jan,148.2,238.1,112.8,–,141.4,167,240.7,241.2,298.2,...,121.5,–,–,111.2,99.3,100.5,–,102,102.7,105.4
1,Feb,153,245.1,113.7,–,146.1,172.8,247.6,248,295.1,...,121.5,–,–,120.1,97.9,101.4,–,102.3,103.4,105.6
2,Mar,159,256.1,115.1,–,151.6,179.8,259.7,260.6,357.9,...,121.5,–,–,126.1,98,100.9,–,102.7,104,106
3,Apr,160.6,259.2,116.5,–,153.2,181.7,262.3,263.3,345.5,...,122,–,–,126.1,98.2,101.1,–,103.9,104.5,106.5
4,May,164.9,265.3,116.3,–,157.4,186.9,267.8,268.9,359.1,...,122.1,–,–,126.3,98,101.2,–,104,104.4,106.7
5,Jun,166.7,265.3,115.9,100,159.3,189,266.7,268.9,362.3,...,122.9,–,–,126.3,97.4,101.5,–,103.8,104.5,106.9
6,Jul,160.5,256.8,115.7,99.8,153.2,181.6,258.5,260.4,300.3,...,130,–,–,126.1,93.5,101.8,–,103.6,105.1,107.7
7,Aug,158.3,255.4,115,99.5,151,179,257.1,258.9,296.4,...,129.8,–,–,129.5,92.7,101.2,–,102.8,105.6,107.8
8,Sep,156.8,253.3,114.4,99,149.6,177.2,255.3,256.9,299.1,...,131.9,–,–,128.5,93.1,101,–,103.1,106,108
9,Oct,156,251.9,113.3,98.4,148.8,176.4,254.5,256.1,322.3,...,137.1,–,–,128.4,92.7,100.3,–,102,105.2,108.2


In [152]:
final_df = copy.deepcopy(df_bea)
#indices_to_remove = [1, 2, 3, 4, 5, 6, 7, 8]

#columns_to_remove = df_bea.columns[indices_to_remove]
# final_df = df_bea.drop(columns_to_remove, axis=1)

# for i, col in enumerate(df_bea.columns):
#     print(f"{col}: {i}")

final_df

,Month,All commodities,Agricultural commodities,All exports excluding food and fuels (Dec. 2010=100),All exports excluding fuels (June 2022=100),Nonagricultural commodities,"All commodities, excluding computers, & semiconductors (Dec. 2001=100)","Foods, feeds, & beverages","Agricultural foods, feeds & beverages, excluding distilled beverages",Wheat and rice,...,"Toys, shooting and sporting goods","Coins, gems, jewelry & collectibles (Dec. 2022=100)",Jewelry (Dec. 2022=100),Nonmanufactured consumer goods (Dec. 2018=100),Biotechnology (Dec. 2021=100),Life Science (Dec. 2019=100),Opto-Electronics (Dec. 2022=100),Information & Communications (Dec. 2019=100),Electronics (Dec. 2019=100),Aerospace (Dec. 2019=100)
0,Jan,148.2,238.1,112.8,–,141.4,167,240.7,241.2,298.2,...,121.5,–,–,111.2,99.3,100.5,–,102,102.7,105.4
1,Feb,153,245.1,113.7,–,146.1,172.8,247.6,248,295.1,...,121.5,–,–,120.1,97.9,101.4,–,102.3,103.4,105.6
2,Mar,159,256.1,115.1,–,151.6,179.8,259.7,260.6,357.9,...,121.5,–,–,126.1,98,100.9,–,102.7,104,106
3,Apr,160.6,259.2,116.5,–,153.2,181.7,262.3,263.3,345.5,...,122,–,–,126.1,98.2,101.1,–,103.9,104.5,106.5
4,May,164.9,265.3,116.3,–,157.4,186.9,267.8,268.9,359.1,...,122.1,–,–,126.3,98,101.2,–,104,104.4,106.7
5,Jun,166.7,265.3,115.9,100,159.3,189,266.7,268.9,362.3,...,122.9,–,–,126.3,97.4,101.5,–,103.8,104.5,106.9
6,Jul,160.5,256.8,115.7,99.8,153.2,181.6,258.5,260.4,300.3,...,130,–,–,126.1,93.5,101.8,–,103.6,105.1,107.7
7,Aug,158.3,255.4,115,99.5,151,179,257.1,258.9,296.4,...,129.8,–,–,129.5,92.7,101.2,–,102.8,105.6,107.8
8,Sep,156.8,253.3,114.4,99,149.6,177.2,255.3,256.9,299.1,...,131.9,–,–,128.5,93.1,101,–,103.1,106,108
9,Oct,156,251.9,113.3,98.4,148.8,176.4,254.5,256.1,322.3,...,137.1,–,–,128.4,92.7,100.3,–,102,105.2,108.2


In [151]:
commodity_prices

,Date,Cattle Hide (Chicago),Maize (US),Natural Gas (US),Soybeans (US),Wheat (US)
0,1995M01,90.12,109.10,33.25,243.00,159.80
1,1995M02,91.42,108.10,34.13,238.00,155.20
2,1995M03,97.95,110.30,33.64,246.00,149.80
3,1995M04,102.32,111.60,34.89,249.00,151.00
4,1995M05,99.61,113.60,35.18,250.00,162.70
...,...,...,...,...,...,...
343,2023M08,NaN,215.05,93.08,583.93,316.52
344,2023M09,NaN,233.20,95.23,619.04,311.37
345,2023M10,NaN,227.60,114.10,529.57,296.41
346,2023M11,NaN,212.44,110.37,553.05,281.60
